# Dimensionality reduction

Let's recap some of the general techniques we've discussed for building better predictive models:
* Properly validating our models is essential if we want to avoid overfitting and underfitting;
* Increasing dataset size is often the single most effective way to improve performance (when feasible); and,
* Understanding the inductive biases of different regularization methods can help us pick good estimators.

To this toolkit, we now add another broad strategy: picking better features. This is, of course, easier said than done. To a large extent, the process of introducing good features—often referred to as *feature engineering*—is a domain-specific enterprise. This makes it hard to describe general methods for engineering good features: if you don't know anything about personality psychology, for example, you may have trouble deciding which items or scales to include in your surveys.

There is, however, one particular approach to feature set construction that can be approached fairly generically: *dimensionality reduction*. In this section, we'll explore strategies for improving predictions by reducing the size of our feature set in various ways.

As always, we begin with some imports...

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

from support import get_features, plot_learning_curves

%matplotlib inline

In [2]:
data = pd.read_csv('data/Johnson-2014-IPIP-300.tsv.gz', sep='\t')

Recall the tension between dataset size and model complexity that's been a recurring theme throughout this tutorial. As we've seen, when we have a limited amount of data, less complex models often outperform more complex ones, because the former are less likely to overfit.

This observation leads directly to another important insight: if we have a feature space that's too big for our model to efficiently learn from, we may be able to reduce the dimensionality of that space until we reach a point that our model is comfortable with. In effect, we're searching for a subspace of our original space that captures most of the reliable variation while shedding most of the noise.

As a toy example, suppose we have a dataset containing three personality scales—the facets of `Vulnerability`, `Anxiety`, and `Excitement-Seeking`. When we inspect the data, we observe that two out of the three scales (`Vulnerability` and `Anxiety`) are strongly correlated:


In [3]:
data[['Anxiety', 'Vulnerability', 'Excitement-Seeking']].corr().round(2)

,Anxiety,Vulnerability,Excitement-Seeking
Anxiety,1.00,0.80,-0.22
Vulnerability,0.80,1.00,-0.11
Excitement-Seeking,-0.22,-0.11,1.00


Or, visually:

In [4]:
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

anx, vul, exc = get_features(data, 'Anxiety', 'Vulnerability', 'Excitement-Seeking', n=500)
from ipywidgets import interact

def plot_3d_scatter(elev, azim):
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(anx, vul, exc, s=20, marker='^')
    ax.set(xlabel='Anxiety', ylabel='Vulnerability', zlabel='Excitement-Seeking')
    ax.view_init(elev, azim);

interact(plot_3d_scatter, elev=45, azim=45);

interactive(children=(IntSlider(value=45, description='elev', max=135, min=-45), IntSlider(value=45, descripti…

When two or more features share variance, it's likely (though by no means guaranteed) that they also share predictively useful variance. Consequently, we may be able to reduce the dimensionality of the feature space, or of some subset of the space, without sacrificing much in the way of predictive power.

Broadly speaking, there are two approaches we could take. First, we could simply select a subset of existing features, without any further manipulation. Second, we could combine some or all of our features to create a smaller number of derivative features.